In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os
import tensorflow_hub as hub

In [7]:
#Downloading datasts form tfds
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name='imdb_reviews',
    split=['train[:90%]','train[90%:]','test'],
    as_supervised=True
)

In [8]:
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000,seed=42).batch(42).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [9]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode('utf-8'))
    print("Label:",label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development

2025-03-23 12:44:28.544075: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-03-23 12:44:28.552689: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-03-23 12:44:28.552891: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews,labels:reviews))

In [ ]:
# Creating Model and training it # with padding which will make model hard to learn and decrease the accuracy
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size,embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])

history = model.fit(train_set,validation_data = valid_set,epochs = 3)

In [ ]:
# Creating the same model using mask_zero = True  = this will make the model to ignore the padding tokens which were added during textvectorization  
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size,embed_size,mask_zero = True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])

history = model.fit(train_set,validation_data = valid_set,epochs = 3)

In [ ]:
# using built in keras ragged tensor that removes padding and more
vocab_size = 1000
text_vec_layer_ragged = tf.keras.layers.TextVectorization(max_tokens=vocab_size,ragged = True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews,labels:reviews))

# Creating Model and training it
embed_size = 128
tf.random.set_seed(42)
model_ragged = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size,embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation='sigmoid')
])


model_ragged.compile(loss = 'binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])

history = model_ragged.fit(train_set,validation_data = valid_set,epochs = 3)
# CONCLUSITN IT IS NOT WORKING PROPERLY SAYS GRU LSTM DOESNOT SUPPORT RAGGED BUT IT WAS MENTIONED IN THE BOOK

# Reusing Pretrained Embeddings and Language Models

In [3]:
# Define the input layer for text data
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

# Load the Universal Sentence Encoder (USE) model from TensorFlow Hub
os.environ['TFHUB_CACHE_DIR'] = 'my_tfhub_cache'

use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=True)

# Wrap the USE layer call within a Lambda layer, specifying output_shape
embedding = tf.keras.layers.Lambda(lambda x: use_layer(x), output_shape=(512,))(text_input)

# Add additional layers (e.g., Dense layers for further processing)
x = tf.keras.layers.Dense(64, activation='relu')(embedding)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Define the complete model using Keras' functional API
model = tf.keras.Model(inputs=text_input, outputs=output)

# Compile the model
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to check architecture
model.summary()

2025-03-23 12:42:36.009905: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,897 (128.50 KB)

 Trainable params: 32,897 (128.50 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(train_set,validation_data= valid_set,epochs = 10)

Epoch 1/10


2025-03-23 12:44:54.194463: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


536/536 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.7869 - loss: 0.4778 - val_accuracy: 0.8496 - val_loss: 0.3315
Epoch 2/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8577 - loss: 0.3330 - val_accuracy: 0.8508 - val_loss: 0.3262
Epoch 3/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8619 - loss: 0.3241 - val_accuracy: 0.8536 - val_loss: 0.3246
Epoch 4/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 22s 41ms/step - accuracy: 0.8653 - loss: 0.3207 - val_accuracy: 0.8528 - val_loss: 0.3211
Epoch 5/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.8686 - loss: 0.3149 - val_accuracy: 0.8548 - val_loss: 0.3200
Epoch 6/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 23s 43ms/step - accuracy: 0.8682 - loss: 0.3143 - val_accuracy: 0.8560 - val_loss: 0.3173
Epoch 7/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - accuracy: 0.8701 - loss: 0.3085 - val_accuracy: 0.8492 - val_loss: 0.3256
Epoch 8/10
536/536 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - accuracy: 0.8709 - loss: 0.3064 - val_accurac